Bril
llvm
xdsl

nanopass

vectorization

mini python 

https://github.com/IUCompilerCourse/Essentials-of-Compilation
https://github.com/IUCompilerCourse/python-student-support-code

https://github.com/IUCompilerCourse/python-student-support-code/blob/main/interp_x86/parser_x86.py


https://github.com/cslarsen/minijit
https://github.com/zserge/tinylangs/blob/main/asm.py
https://github.com/benhoyt/pyast64  https://benhoyt.com/writings/pyast64/
https://github.com/Maratyszcza/PeachPy
https://github.com/ssloy/tinycompiler

https://news.ycombinator.com/item?id=14644576

https://gitlab.com/tsoding/porth https://www.youtube.com/watch?v=QlKSCn-Al6Q&ab_channel=TsodingDaily


Some mix of meta and internalized

https://github.com/slothy-optimizer/slothy/ https://eprint.iacr.org/2022/1303

In [17]:
from kdrag.all import *
BV64 = smt.BitVecSort(64)
getarg = smt.Function("getarg", smt.IntSort(), BV64)

def remove_complex_operands(e : smt.ExprRef):
    todo = [e]
    prog = []
    while todo:
        e = todo.pop()
        prog.append(e)
        if isinstance(e, smt.BitVecNumRef):
            continue
        if smt.is_app(e):
            if e.decl() != getarg:
                assert e.num_args() <= 2
                todo.extend(e.children())
        else:
            raise ValueError("Expected an application, got: ", e)
    return list(reversed(prog))
prog = getarg(0) + 42 - getarg(1)
prog1 = remove_complex_operands(prog)

In [ ]:
def remove_complex_correct(prog0, prog1):
    # There's nothing much to say here.
    pass

In [ ]:
movq = smt.Function("movq", BV64, BV64)
subq = smt.Function("subq", BV64, BV64, BV64)
addq = smt.Function("addq", BV64, BV64, BV64)
Reg = kd.Enum("Reg", ["RAX", "RBX", "RCX", "RDX", "RSI", "RDI", "RSP", "RBP", "R8", "R9", "R10", "R11", "R12", "R13", "R14", "R15"])
Insn = kd.Enum("Insn", ["movq", "addq", "subq"])

def instruction_select(prog : list[smt.ExprRef]):
    res = []

    for expr in prog:
        if isinstance(expr, smt.BitVecNumRef):
            continue
        else:
            match expr.decl().name():
                case "getarg":
                    i = expr.arg(0)
                    assert isinstance(i, smt.IntNumRef)
                    reg = [Reg.RDI, Reg.RSI, Reg.RDX, Reg.RCX, Reg.R8, Reg.R9][i.as_long()]
                    res += [(movq, reg, expr)]
                case "bvadd":
                    arg1,arg2 = expr.children()
                    res += [(movq, arg1, Reg.RAX),
                            (addq, arg2, Reg.RAX),
                            (movq, Reg.RAX, expr)]
                case "bvsub":
                    arg1,arg2 = expr.children()
                    res += [(movq, arg1, Reg.RAX),
                            (subq, arg2, Reg.RAX),
                            (movq, Reg.RAX, expr)]
                
                case _:
                    raise ValueError("Unexpected expression in instruction selection: ", expr)
    return res


prog2 = instruction_select(prog1)
prog2 

[(movq, RDI, getarg(0)),
 (movq, getarg(0), RAX),
 (addq, 42, RAX),
 (movq, RAX, getarg(0) + 42),
 (movq, RSI, getarg(1)),
 (movq, getarg(0) + 42, RAX),
 (subq, getarg(1), RAX),
 (movq, RAX, getarg(0) + 42 - getarg(1))]

In [ ]:
rsp = smt.BitVec("rsp", 64)

def assign_home(prog):
    res = []
    stack = {}
    for stmt in prog:
        match stmt:
            case (op, arg1, arg2):
                print(f"{op} {arg1} {arg2}")
                if arg1.sort() == BV64:
                    if arg1 not in stack:
                        stack[arg1] = rsp + len(stack) * 8
                res.append((op, stack[arg1], stack[arg2]))
            case _:
                raise ValueError("Unexpected statement in assignment: ", stmt)
    return res
assign_home(prog2)

movq RDI getarg(0)
movq getarg(0) RAX
addq 42 RAX
movq RAX getarg(0) + 42
movq RSI getarg(1)
movq getarg(0) + 42 RAX
subq getarg(1) RAX
movq RAX getarg(0) + 42 - getarg(1)


In [ ]:
    env = {}
    regfile = smt.Const("undefined", Regfile)
regfile[[RDI, RDX][i.as_long()]]
Regfile = smt.ArraySort(Reg, BV64)

mov_to_rax = smt.Function("movq", BV64, Regfile, Regfile)
read_rax = smt.Function("read_rax", Regfile, BV64)
add_rax = smt.Function("add_rax", BV64, Regfile, Regfile)




In [ ]:
Stmt = smt.ArraySort(State, State)
RR = smt.Function("RR", Insn, Reg,   Reg,       State, State) # Reg Reg insn
smt.Function("MR", Insn, Reg, BV64,  Reg,  State, State) # Mem Reg insn
smt.Function("IR", Insn, BV64,   Reg,      State, State) # Imm Reg insn
smt.Function("RM", Insn, Reg,  Reg, BV64,  State, State) # Reg Reg insn

def from_tup(insn):
    insn, op1, op2 = insn
    s1,s2 = op1.sort(), op2.sort()
    if s1 == Reg and s2 == Reg:
        return RR(insn, op1, op2)
    elif s1 == Reg and s2 == BV64:
        return MR(insn, op1, op2)
    elif s1 == BV64 and s2 == Reg:
        return RM(insn, op1, op2)



In [ ]:
Stack = smt.ArraySort(smt.BitVecSort(8), BV64)
Reg = kd.Enum("Reg", ["RAX", "RBX", "RCX", "RDX", "RSI", "RDI", "RSP", "RBP"])
Regfile = smt.ArraySort(Reg, BV64)
movqrr = smt.Function("movqrr", Reg, Reg,  Regfile, Regfile)
movqlr = smt.Function("movqlr", BV64, Reg, Regfile, Regfile)
movqrl = smt.Function("movqrl", BV64, Reg, Regfile, Regfile)
movqsr = smt.Function("movqsr", Reg, Stack, Regfile, Regfile)
movqrs = smt.Function("movqrs", Stack, Reg, Regfile, Stack)

State = kd.Struct("x86State", ("regs", Regfile), ("stack", Stack))


TypeError: Enum() takes 2 positional arguments but 9 were given

target interpreter



In [ ]:
%%file /tmp/test.S

.globl mytest
mytest:
    mov $42, %rax
    ret

In [ ]:
!gcc -c -fPIC -o /tmp/test.o /tmp/test.S && ld -shared -o /tmp/test2.so /tmp/test.o && objdump -d /tmp/test.so

In [ ]:
import cffi

ffi = cffi.FFI()
ffi.cdef("""uint64_t mytest();""")
lib = ffi.dlopen("/tmp/test2.so")
dir(lib)
print(lib.mytest())
del lib

def 

# Interactive Assembly

Typed assembly language. I've been saying that type systems can be seen as tactics.
https://dl.acm.org/doi/10.1007/978-3-031-74776-2_17 BinSub: The Simple Essence of Polymorphic Type Inference for Machine Code

webassembly via ghidra? https://github.com/nneonneo/ghidra-wasm-plugin/


dump into dwarf?
https://github.com/pq-code-package/mlkem-native/tree/main/proofs/hol_light/arm 
https://github.com/awslabs/s2n-bignum
https://github.com/awslabs/s2n-bignum/blob/main/x86/tutorial/simple.ml


global invariants.
Interrupt_enabled => 


Python smt assertions. just use eval. If smtlib turns you off
Dwarf expr assertions
assembly subprogram assertiosn - the most work. Natural in some sense.




In [ ]:
%%file /tmp/myfunc.s

.macro kd_assert label smt_expr
\label:
.endm

.macro kd_pre label smt_expr
.endm


.macro kd_post label smt_expr
.endm

# kd_entry?


# myfunc.s
.globl myfunc

.text
myfunc:
    kd_pre myfunc "(assert true)"
    kd_post myfunc "(assert (= RAX (_ bv42 64)))"
    movq $42, %rax
    kd_assert func_end "(assert (= RAX (_ bv42 64)))"
    ret


Overwriting /tmp/myfunc.s


In [38]:
! as -o /tmp/myfunc.o /tmp/myfunc.s && objdump -d /tmp/myfunc.o


/tmp/myfunc.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <myfunc>:
   0:	48 c7 c0 2a 00 00 00 	mov    $0x2a,%rax

0000000000000007 <func_end>:
   7:	c3                   	ret


In [ ]:
import re
from kdrag.all import *
def find_kdasserts(lines):
    pattern = re.compile(r'^\s*kd_assert\s+(\w+)\s+"([^"]+)"\s*$')
    results = []
    for lineno, line in enumerate(lines, 1):
        match = pattern.match(line)
        if match:
            label, smt_expr = match.groups()
            RAX = smt.BitVec("RAX", 64)
            smt_expr = smt.parse_smt2_string(smt_expr, decls={"RAX": RAX})
            results.append((lineno, label, smt_expr))
    return results

# Example usage:

with open("/tmp/myfunc.s") as f:
    lines = f.readlines()
    asserts = find_kdasserts(lines)
    print(asserts)



import kdrag.contrib.pcode as pcode
from kdrag.all import *
ctx = pcode.BinaryContext("/tmp/myfunc.o")
#ctx.loader.symbols
for (lineno, label, smt_expr) in asserts:
    sym = ctx.loader.find_symbol(label)
    print(sym.name, sym.rebased_addr)

myfunc_sym = ctx.loader.find_symbol("myfunc")
mem = pcode.MemState.Const("mem")
symstates = ctx.sym_execute(mem, myfunc_sym.rebased_addr, max_insns=1, verbose=True)
smt.simplify(symstates[0].memstate.getvalue(ctx.ctx.registers["RAX"]))

mem = symstates[0].memstate
#{smt.BitVec(name, vnode.size*8) : smt.simplify(mem.getvalue(vnode)) for name,vnode in ctx.ctx.registers.items()}

def get_reg(ctx, mem, name):
    vnode = ctx.ctx.registers[name]
    return smt.BitVec(name, vnode.size*8), smt.simplify(mem.getvalue(vnode))

get_reg(ctx, mem, "RAX")

decls = [smt.BitVec(name, vnode.size * 8) for name, vnode in ctx.ctx.registers.items()]
decls.append(smt.Array("mem", smt.BitVecSort(64), smt.BitVecSort(8)))
# select32
# select64
# old_mem? ghost state?  kd_ghost "(define)"    kd_assign  ghostname "smt expr"
# yikes.

decls # decls is for parsing 

[(22, 'func_end', [RAX == 42])]
func_end 4194311
Executing 0x400000/7: MOV RAX,0x2a at (4194304, 0)
Executing 0x400000/7: MOV RAX,0x2a at (4194304, 1)


[RAX,
 EAX,
 AX,
 AL,
 AH,
 RCX,
 ECX,
 CX,
 CL,
 CH,
 RDX,
 EDX,
 DX,
 DL,
 DH,
 RBX,
 EBX,
 BX,
 BL,
 BH,
 RSP,
 ESP,
 SP,
 SPL,
 RBP,
 EBP,
 BP,
 BPL,
 RSI,
 ESI,
 SI,
 SIL,
 RDI,
 EDI,
 DI,
 DIL,
 R8,
 R8D,
 R8W,
 R8B,
 R9,
 R9D,
 R9W,
 R9B,
 R10,
 R10D,
 R10W,
 R10B,
 R11,
 R11D,
 R11W,
 R11B,
 R12,
 R12D,
 R12W,
 R12B,
 R13,
 R13D,
 R13W,
 R13B,
 R14,
 R14D,
 R14W,
 R14B,
 R15,
 R15D,
 R15W,
 R15B,
 ES,
 CS,
 SS,
 DS,
 FS,
 GS,
 FS_OFFSET,
 GS_OFFSET,
 CF,
 F1,
 PF,
 F3,
 AF,
 F5,
 ZF,
 SF,
 TF,
 IF,
 DF,
 OF,
 IOPL,
 NT,
 F15,
 RF,
 VM,
 AC,
 VIF,
 VIP,
 ID,
 rflags,
 eflags,
 flags,
 RIP,
 EIP,
 IP,
 DR0,
 DR1,
 DR2,
 DR3,
 DR4,
 DR5,
 DR6,
 DR7,
 DR8,
 DR9,
 DR10,
 DR11,
 DR12,
 DR13,
 DR14,
 DR15,
 CR0,
 CR1,
 CR2,
 CR3,
 CR4,
 CR5,
 CR6,
 CR7,
 CR8,
 CR9,
 CR10,
 CR11,
 CR12,
 CR13,
 CR14,
 CR15,
 XCR0,
 BNDCFGS,
 BNDCFGU,
 BNDSTATUS,
 BND0,
 BND0_LB,
 BND0_UB,
 BND1,
 BND1_LB,
 BND1_UB,
 BND2,
 BND2_LB,
 BND2_UB,
 BND3,
 BND3_LB,
 BND3_UB,
 SSP,
 IA32_PL2_SSP,
 IA32_PL1_SSP

Use angr to get cfg
Symexec blocks into semantics.

How does angr do translation between registers and stuff

Difference between an assertion and a cut / invariant. (?)


Ideas: embed python script or bash commands to grep manuals for assumptions.
Embed Verilog assertions? Read from svd files?
Remember that idea about keeping docs in sync. A third yaml file?


Reasoning about interrupts. Interrupt prempting interrupts. Not noticing you need to lock critical sections. How to detect?
Yikes.


Am I insane? What is really better about this than other systems. Is there some off the shlef binary verifier i could use?





In [33]:
import angr
proj = angr.Project("/tmp/myfunc.o", auto_load_libs=False)
cfg = proj.analyses.CFGFast()
print("This is the graph:", cfg.model.graph)
f = cfg.kb.functions[myfunc_sym.rebased_addr]
blk = list(f.blocks)[0] # transition _Graph, black_addrs, 
blk.instructions
blk.addr

for blk in f.blocks:
    res = ctx.sym_execute(mem, blk.addr, max_insns=blk.instructions-1, verbose=True)
    print(res)

dir(blk)

#cfg.kb.functions[sym.rebased_addr]

This is the graph: DiGraph with 1 nodes and 0 edges
Executing 0x400000/7: MOV RAX,0x2a at (4194304, 0)
Executing 0x400000/7: MOV RAX,0x2a at (4194304, 1)
[SimState(memstate=MemState(mem=MemState(ram(mem),
         Store(Store(Store(Store(Store(Store(Store(Store(register(MemState(ram(mem),
                                        Store(Store(Store(Store(Store(Store(Store(Store(register(mem),
                                        0 + 0,
                                        Extract(7, 0, 42)),
                                        0 + 1,
                                        Extract(15, 8, 42)),
                                        0 + 2,
                                        Extract(23, 16, 42)),
                                        0 + 3,
                                        Extract(31, 24, 42)),
                                        0 + 4,
                                        Extract(39, 32, 42)),
                                        0 + 5,
                  

['BLOCK_MAX_SIZE',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_bytes',
 '_capstone',
 '_collect_data_refs',
 '_const_prop',
 '_cross_insn_opt',
 '_disassembly',
 '_get_cmsg',
 '_initial_regs',
 '_instruction_addrs',
 '_instructions',
 '_load_from_ro_regions',
 '_opt_level',
 '_parse_vex_info',
 '_project',
 '_strict_block_end',
 '_using_pcode_engine',
 '_vex',
 '_vex_engine',
 '_vex_nostmt',
 'addr',
 'arch',
 'bytes',
 'capstone',
 'codenode',
 'disassembly',
 'instruction_addrs',
 'instructions',
 'parse',
 'parse_from_cmessage',
 'pp',
 'reset_initial_regs',
 'serialize',
 'serialize_to_cmessage',
 'set_initial_regs',
 'size',
 'thumb',
 've

def get_asserts(lines):
    pattern = r'kdassert\s+(\w+)\s+"([^"]+)"'
    matches = re.findall(pattern, text)
    return matches:


with open("myfunc.s") as f:
    get_asserts(f.readlines())